In [1]:
import sys
sys.path.append('..')

In [2]:
from agents.dqn import *
from core.replay import *
from environments.wrappers import *
from networks.cnn import *
from utils.train import *
from utils.logger import *

In [3]:
import gym
import torch

### Network, Environment, Agent

In [4]:
network = DQNCNN()

In [5]:
# parameters
size = (84, 84)
num_stacked_frames = 4

raw_env = gym.make('BreakoutNoFrameskip-v4')
env = AtariWrapper(raw_env, size=size, k=num_stacked_frames)

In [6]:
observation_space = raw_env.observation_space
action_space = raw_env.action_space
params = {'epsilon':1.0, 'epsilon_decay':0.99, 'epsilon_min':0.05, 'gamma':0.99, 'alpha':2.5e-5,
          'memory_size':250000, 'device':'cuda', 'network':network, 'batch_size':32, 'target_net_updates':10000}

agent = DQNAgent(observation_space, action_space, **params)

### Train

In [7]:
logger = Logger('training_info')

In [8]:
training_params = {'num_steps': 0, 'total_steps':20000000, 'logger':logger, 'save_freq':1000, 'e_verbose':100, 'file_name': 'atari ddqn'}

In [ ]:
standard_train(agent, env, **training_params)